In [16]:
! ollama pull gemma3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aeda25e63ebd: 100% ▕██████████████████▏ 3.3 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling b6ae5839783f: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [17]:
! ollama pull nomic-embed-text


pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90: 100% ▕██████████████████▏ 274 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling ce4a164fc046: 100% ▕██████████████████▏   17 B                         
pulling 31df23ea7daa: 100% ▕██████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [18]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [19]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema.runnable import RunnableLambda

In [20]:
import gradio as gr

In [21]:
import glob

In [22]:
llm = Ollama(model="gemma3", base_url="http://127.0.0.1:11434",temperature = 0.2)

In [23]:
embed_model = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://127.0.0.1:11434"
)


In [24]:
from langchain_community.document_loaders import PyPDFLoader
import os
from langchain.document_loaders import DirectoryLoader, TextLoader

In [25]:
def file_process(files):
    system_prompt = "You are a helpful assistant that provides concise and accurate responses based on provided documents."
    system_prompt += "If the customer greets with Hi or hello greet them and if the customer talks anything in general respond based on his message."
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_prompt),
        HumanMessagePromptTemplate.from_template("{question}")
    ])

    documents = []

    for file in files:

        file_path = getattr(file, 'name', None) or getattr(file, 'tempfile', None)
    
        if not file_path:
            raise ValueError("Invalid file upload")
            
        # Load and process PDF
        loader = PyPDFLoader(file_path)
        docs = loader.load()
            
        # Add filename to metadata
        for doc in docs:
            doc.metadata['filename'] = os.path.basename(file.name).split(".")[0]
            documents.append(doc)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    chunks = text_splitter.split_documents(documents)
    for i in chunks:
        i.page_content = 'this chunk is from' +" "+ i.metadata['filename']+":" +" "+ i.page_content 
    db_name = 'vector_db'
    
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name, embedding_function=embed_model).delete_collection()
        
    vector_store = Chroma.from_documents(documents = chunks, embedding = embed_model, persist_directory=db_name)
    collection = vector_store._collection
    retriever = vector_store.as_retriever(search_type="mmr",  # Maximal Marginal Relevance
    search_kwargs={"k": 5, "fetch_k": 20})
    llm_with_prompt = prompt | llm  # Automatically formats messages then runs LLM
    memory = ConversationBufferMemory(memory_key= 'chat_history', return_messages = True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm_with_prompt,
        retriever=retriever,
        memory=memory,
        return_source_documents=False
        )
    return conversation_chain

    
    

In [26]:
def chat_func(messsage, history):
    conversation_chain = file_process(file)
    

In [27]:
def upload_func(files):
    global current_conversation_chain
    conversation_chain = file_process(files)
    if conversation_chain:
        current_conversation_chain = conversation_chain
    return 'File uploaded successfully'

In [28]:

# Chat function
def chat_func(message, history):
    global current_conversation_chain
    if current_conversation_chain is None:
        return "Please upload documents first before asking questions."
    
    result = current_conversation_chain.invoke({'question': message})
    return result['answer']

In [29]:
with gr.Blocks(css="""
    .gradio-container, .gradio-container > div {
        height: 100vh !important;
    }
    .chat-column {
        height: calc(100vh - 140px);
    }
    .file-column {
        height: 50%;
    }""") as demo:
    gr.Markdown("# RAG Chatbot 🤖")
    gr.Markdown("Upload your PDF documents and ask questions about them.")
    
    with gr.Row():
        with gr.Column(scale = 1):
            upload_button = gr.File(
                label = "Click to Upload PDFs", 
                file_types=[".pdf"], 
                file_count="multiple"
            )
            file_output = gr.Textbox(label="Upload Status")
            upload_button.upload(
                upload_func,
                inputs=upload_button,
                outputs=file_output
            )
        with gr.Column(scale = 2):
            chatbot = gr.ChatInterface(
                chat_func,
                chatbot=gr.Chatbot(height=500),
                #textbox=gr.Textbox(placeholder="Ask questions about your documents...", container=False)
            )

/var/folders/z1/2kj6d1ld1w5f46gd1x6cqh2m0000gn/T/ipykernel_32038/1259904431.py:30: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


In [30]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/var/folders/z1/2kj6d1ld1w5f46gd1x6cqh2m0000gn/T/ipykernel_32038/1198931100.py:33: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  Chroma(persist_directory=db_name, embedding_function=embed_model).delete_collection()
/var/folders/z1/2kj6d1ld1w5f46gd1x6cqh2m0000gn/T/ipykernel_32038/1198931100.py:40: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key= 'chat_history', return_messages = True)
Number of requested results 20 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 20 is greater than number of elements in index 4, updating n_resul